In [0]:
%pip install sqlglot
%pip install langchain_core
%pip install nbformat

In [0]:
dbutils.library.restartPython()

In [0]:
import sqlglot
import os

In [0]:
dbutils.widgets.text("input_sql_dir1", "/Workspace/Users/pxchall2@sentara.com/InPut_SPs", "The location of the SQL files to be converted")
dbutils.widgets.text("output_dir1", "/Workspace/Users/pxchall2@sentara.com/OutPut_SPs", "The output location for the notebooks")


In [0]:
import os

# Get the paths from the widgets
proc_dir = dbutils.widgets.get("input_sql_dir1")
output_folder = dbutils.widgets.get("output_dir1")

# List files in the DBFS directory
procs = os.listdir(proc_dir)
procs.sort()
procs

In [0]:
import chardet
with open(proc_dir + "/" + procs[0], 'rb') as file:
    raw = file.read(32) 
    encoding = chardet.detect(raw)['encoding']

print(encoding)

In [0]:
from pathlib import Path

parsed_code = {}
for procedure_name in procs:
  with open(os.path.join(proc_dir, procedure_name), encoding=encoding) as content_file:
    code = content_file.read()
    try:
      print("parsing code for {0}".format(procedure_name))
      parsed_code[procedure_name] = sqlglot.parse(code, read="tsql")
    except Exception as e:
      print(f"Unable to parse {procedure_name} - {e}")

In [0]:
len(parsed_code)

In [0]:
for k,v in parsed_code.items():
  print(type(v))
  print(v)

In [0]:
declare_statement = ""
for procedure_name,code in parsed_code.items():
  for item in code:
    declarations = item.find_all(sqlglot.exp.DeclareItem)
    for declaration in declarations:
      print(declaration)
      declare_statement += "DECLARE {0} {1};".format(declaration.this.name, declaration.args["kind"].sql())
print(declare_statement)

In [0]:
statements = []
i = 0
for procedure_name,code in parsed_code.items():
  for s in code:
    # if (isinstance(s, sqlglot.expressions.Create)) or (isinstance(s, sqlglot.expressions.Declare)):
    statements.append(s)
    i = i + 1

In [0]:
len(statements)

In [0]:
for c in statements:
  print(type(c))